In [22]:
import folium
import pandas as pd
import numpy as np

# The following URL provides daily updates on coronavirus cases/deaths in Michigan.
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163-520743--,00.html'

# The attached file has the geo locations of each county in Michigan.
filename = 'CountyLocations.csv'


city = pd.read_csv(filename)
city = city.iloc[:,:3]
city.iloc[:,0] = city['County'].str.replace('Michigan', '', regex=False)

data = pd.read_html(url,header=0)
data = data[0].iloc[:-3,:].fillna(0)
data['Lats'] = np.zeros(len(data))
data['Longs'] = np.zeros(len(data))
data = data.rename(columns={"Reported Deaths": "Deaths"})
# data

In [23]:
# city

In [24]:
for (index,county) in data.iterrows():
    for (i,c) in city.iterrows():
        if county[0] == c[0]:
            data.iloc[index,3] = city.iloc[i,1]
            data.iloc[index,4] = city.iloc[i,2]

In [25]:
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster, MarkerCluster

geo_locs = np.array(data[["Lats","Longs"]])
cases = np.array(data.iloc[:,1])
deaths = np.array(data.iloc[:,2])

map = folium.Map(location=[43.740957, -84.710854],zoom_start=7)

mc = MarkerCluster(name="Marker Cluster")

heat_frame = []
for i in range(len(data)):
    heat_frame.append(np.tile(geo_locs[i],(cases[i],1)))

heat_data = heat_frame[0]
for i in heat_frame[1:]:
    heat_data = np.append(heat_data,i,axis=0)
HeatMap(heat_data).add_to(map);

data['Deaths'] = data.Deaths.astype(float)
    
mc = MarkerCluster()
for row in data.itertuples():
    mc.add_child(folium.Marker(location=[row.Lats,  row.Longs],
                 popup=str(row.County)+"<br>Cases:"+str(row.Cases)+"<br>Deaths:"+str(row.Deaths)))
    map.add_child(mc)
    
mc.add_to(map)
folium.LayerControl().add_to(map);

In [26]:
map